# Aqarios Luna Platform Demo

This jupyter notebook will walk you through some of the potential uses of the Luna platform with the goal of demonstrating how this tool can be integrated into existing workflows. 

This demo will only include functionality from LunaSolve, our platform focused on optimization using state-of-the-art quantum, hybrid, and classical heuristics.

The structure of this notebook is as follows:
 - **Part 1**: Setup
 - **Part 2**: Real-world optimization using built-in transformations
 - **Part 3**: Open-ended optimization using manuallu constructed QUBOs

For a complete overview of our API's functionality visit https://api.aqarios.com/docs for interative documentation.

## Part 1: Setup

In order to make this notebook interactive, there are a few necessary steps to setup our environment.

### A. Install the necessary packages

In [ ]:
%pip install requests
%pip install networkx

### B. Set the API URL for requests

In [ ]:
URL = "https://api.aqarios.com"

### C. Authenticate your account

Create a file in this same directory named: `credentials.json`.

This will house your secret login information so that you can request access tokens from the platform.

The file should look like this:

```
{
    "username": "YOUREMAILHERE",
    "password": "YOURPASSWORDHERE"
}
```

In [ ]:
import json
import requests

In [ ]:
# get credentials from secret file
with open("credentials.json", 'r') as f:
    credentials = json.load(f)
    username = credentials['username']
    password = credentials['password']

# request API for token using credentials
response = requests.post(
    url=URL+"/accessToken",
    data={
        "username": username,
        "password": password
    }
)

# set received token as header
token = response.json().get("access_token")
headers = {"Authorization": f"Bearer {token}"}

## Part 2: Real-world optimization

There are many computationally challenging optimization problems defined in the literature, which cover a variety of domains. Examples like the Traveling Salesman Problem, Job-Shop Scheduling, or the Knapsack Problem are likely familiar to many who have spent time in the field. 

Many, but not all, of these optimization problems have been analized by researchers interested in attempting to solve them of quantum hardware. This process requires a transformative step. 

While the Traveling Salesman Problem is described by a graph which contains all the desired destinations and the distances between them, this representation is not suitable for current quantum annealer, let alone classical optimization heuristics. 

This is the first functionaly of LunaSolve: **Transform**. 

We provide Users with a built-in library of (currently) **40** different optimization problems which can be configured using human-readable data formats (such as graphs). These are then automatically transformed into appropriate mathematical represetations for quantum computers.

### A. Retrieve available optimizations

The list of currently available optimizations can be retrieved using the `/api/problem/info/all` endpoint. However, we recommend browsing this information using the interactive documentation referenced above.

Here is an example of an optimization specification from the docs:

<div>
<img src="static/tsp-docs.jpg" width="800"/>
</div>

This includes a description of the optimization problem, its attributes, and references to the publication that its transformation is taken from.

The full list can be found by:
1. Navigating to https://api.aqarios.com/docs#/Optimizations/create_optimization_api_optimizations_post
2. Clicking on **Schema** in the Request Body section.
3. Opening the `instance` dropdown.
4. Opening the optimization problem of your choice.


### B. Create an instance of an optimization

Suppose you need to coordinate the shipping of wholesale products to across Germany. You have retailers in the seven largest German ciites and want to deliver all of your product at once using the shortest route possible to minize transportation costs.

For this problem we have found the distances between each of these major cities, and now need to format this data in terms of a graph with edge weights. 

In [ ]:
import networkx as nx

In [ ]:
# Create an empty graph
graph = nx.Graph()

# Add nodes
cities = ['Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt', 'Stuttgart', 'Düsseldorf']
graph.add_nodes_from(cities)

# Add edges and their weights (distances between cities)
edges = [('Berlin', 'Hamburg', 289), 
         ('Berlin', 'Munich', 590), 
         ('Berlin', 'Cologne', 579), 
         ('Berlin', 'Frankfurt', 551), 
         ('Berlin', 'Stuttgart', 639), 
         ('Berlin', 'Düsseldorf', 569), 
         ('Hamburg', 'Munich', 791), 
         ('Hamburg', 'Cologne', 433), 
         ('Hamburg', 'Frankfurt', 493), 
         ('Hamburg', 'Stuttgart', 656), 
         ('Hamburg', 'Düsseldorf', 401), 
         ('Munich', 'Cologne', 573), 
         ('Munich', 'Frankfurt', 392), 
         ('Munich', 'Stuttgart', 233), 
         ('Munich', 'Düsseldorf', 612), 
         ('Cologne', 'Frankfurt', 212), 
         ('Cologne', 'Stuttgart', 377), 
         ('Cologne', 'Düsseldorf', 54), 
         ('Frankfurt', 'Stuttgart', 205), 
         ('Frankfurt', 'Düsseldorf', 229), 
         ('Stuttgart', 'Düsseldorf', 416)]
graph.add_weighted_edges_from(edges)

graph_data = nx.to_dict_of_dicts(graph)

Now send a request to the API to create a new optimization instance according to this graph specification.

In [ ]:
request_body = {
    "instance": {
        "name": "TSP",
        "graph": graph_data
    }
}

response = requests.post(
    url=URL+"/api/optimizations",
    headers=headers,
    json=request_body
)

tsp_id = response.json().get("_id")
f"id: {tsp_id}"

### C. Solve your optimization

Now that you have received a id for your TSP problem, you can make use of the other services LunaSolve provides. 

Before simply calling up some arbitrary solving heuristic, use LunaSolve's *AI-powered* **Recommendation engine** to select the best solver for your particular problem.

*Note: the `/recommend` endpoint is currently still in a prototypical state and as such will only recommend classical solvers. More to come in future updates.*

In [ ]:
response = requests.get(
    url=URL+f"/api/optimizations/{tsp_id}/recommendation",
    headers=headers
)

recommendation = response.json()
recommendation

Now, use the received recommendation to request a solution from the API

In [ ]:
response = requests.post(
    url=URL+f"/api/optimizations/{tsp_id}/solutions",
    headers=headers,
    params={"solver_name": "tabu_search"},
    json=recommendation.get("solver_params")
)

solution_url = response.json().get("solution_url")
solution_url

Since calls to some solvers (quantum hardware in particular) can take an extended period of time of complete, requesting a solution be generated returns a 202 code with a `solution_url` where the final result can be retrieved from the API once the computation is completed.

In [ ]:
response = requests.get(
    url=URL+f"/{solution_url}",
    headers=headers
)

solution = response.json()

In [ ]:
# TODO: translate solution back into problem domain.

## Part 3: Custom QUBOs

As useful as the library of optimization problems is, not all scenarios can be fit cleanly into a pre-defined problem definition. Some users may wish to input and solve their own custom mathematical models. 

Our services currently make all the samy functionalities available to optimization problems available to problems formulated as QUBO (quadratic unconstrained binary optimization) matrices. The matrices are passed as nested lists of floats.

### A. Initialize QUBO

First, create a list of lists of floats (or integers) which represents the QUBO in question. Then send that qubo to LunaSolve.

In [ ]:
qubo = [
    [4, 0, 0, 0, -2],
    [0, -2, 0, 0, 0],
    [0, 0, 6, -3, 0],
    [0, 0, -3, 2, 0],
    [-2, 0, 0, 0, 5]
]

In [ ]:
response = requests.post(
    url=URL+f"/api/qubos",
    headers=headers,
    json={"matrix": qubo}
)

qubo_id = response.json().get("_id")
f"id: {qubo_id}"

### B. Solve QUBO

Request the creation of a new solution for the QUBO and retrieve it once solving is completed.

In [ ]:
response = requests.post(
    url=URL+f"/api/qubos/{qubo_id}/solutions",
    headers=headers,
    params={"solver_name": "simulated_annealing"}
    json={"num_sweeps": 100, "num_reads"= 10}
)

solution = response.json()
solution

### C. Custom Solving Heuristic

Researchers are often wish to develop their own proprietary heuristics for solving QUBOs effectively. These can involve complicated business logic for pre- and post-processing of results. LunaSolve can support developers of optimization heuristics by unifying the access point to both classical and quantum solvers. 

As an example of how Luna's integration could support the development of an innovative approach to QUBO solving, one can imagine a heuristic which strategically splits a QUBO into sub-QUBOs, which are then sent to existing solvers so that the results can be brought together to for an optimal total solution. This kind of functionality can be easily achieved using LunaSolve, as the business logic behind the novel heuristic can be written and maintained in the desired programming language and corresponding repository, while the Luna platform acts as a signel point of access to the solving heuristics which must be called inside the custom algorithm.